In [3]:
import spacy
from spacy import displacy
import os
from graphviz import Digraph
from spacy.vocab import Vocab
from spacy.tokens import Doc
from pathlib import Path

nlp = spacy.load("zh_core_web_sm")

In [10]:
class Conll_8best_Read():
    def __init__(self,dataset):
        self.dataset=dataset
    #这个函数没测试过 不过多个的没问题，这个应该也没啥问题
    def read_single_file(self,file_path,file_name):
        #path = E:\\baiduyunxiazai\\zx
        #file_path = "E:\\baiduyunxiazai\\zx\\top1\\notMatch.1"
        paths=['top1','top2','top3','top4','top5','top6','top7','top8']
        #file_path = "E:\\baiduyunxiazai\\zx\\top1\\notMatch.1"
        #file_path+paths[j]+notMatch.alldata
        file = os.path.join(file_path,paths[0],file_name)
        with open(file,'r', encoding='utf-8') as fb:
            for line in fb:
                line = line.strip('\n') 
                if line:
                    self.dataset.append(line.split('\t'))
                else:
                    self.dataset.append([])
        return self.dataset
         
    def read_mul_file(self,file_path,file_name):
        #一共973句话 算上']'
        #path = E:\\baiduyunxiazai\\zx
        paths=['top1','top2','top3','top4','top5','top6','top7','top8']
        #file_path = "E:\\baiduyunxiazai\\zx\\top1\\notMatch.1"
        for j in range(8):
            #file_path+paths[j]+notMatch.alldata
            file = os.path.join(file_path,paths[j],file_name)
            print(file)
            with open(file,'r', encoding='utf-8') as fb:
                for line in fb:
                    line = line.strip('\n') 
                    if line:
                        self.dataset.append(line.split('\t'))
                    else:
                        self.dataset.append([])
        return self.dataset
         

class Sentence(object):
    def __init__(self):
        self.sen = "" #用来判断句子是否相同
        self.idx= [] #存每一个单词的序号
        self.word = [] #存每一个单词
        self.tag = [] #存储标签
        self.dep = [] #存储依赖
        self.rel = [] #依赖关系
        self.Arc={} #字典
        self.special_name=[]
        
    def equal(self,a,b):
        if a == b:
            return True
        else:
            return False
    def dep_tree(self):
        #这里面把所有的弧都取出来
        for i in range(len(self.word)):
            if self.dep[i] == '0':
                self.Arc[self.special_name[i]] = 'root'
            else:
                for j in range(len(self.word)):
                    if self.idx[j] == self.dep[i]:
                        self.Arc[self.special_name[i]] = self.special_name[j]
                
            


In [43]:
def load_data_conll(dataset):
    Sen = Sentence()
    sente = [Sentence() for i in range(1)]
    for i in range(len(dataset)):
        if dataset[i]:
            Sen.sen = Sen.sen + dataset[i][1]
            Sen.word.append(dataset[i][1])
            Sen.idx.append(dataset[i][0])
            Sen.tag.append(dataset[i][3])
            Sen.dep.append(dataset[i][6])
            Sen.rel.append(dataset[i][7])
            Sen.special_name.append(str(dataset[i][0])+'|'+str(dataset[i][1]+'|'+str(dataset[i][3])))
        else:
            Sen.dep_tree()
            sente.append(Sen)
            Sen = Sentence()
    del(sente[0])
    print(sente[0].sen)
    return sente

def find_sent(sen):
    se = set() #存储所有已经出现了的句子
    dic = {"":[]} #为每个句子存储所有它的index
    for i in range(len(sen)):
        if sen[i].sen not in se:
            dic[sen[i].sen]=[i] #为每个句子建立一个空集合
        else:
            continue #这个句子我们就不统计了 进行一个值的跳过
        
        for j in range(i+1,len(sen)):
            if sen[j].sen not in se and sen[i].sen == sen[j].sen: 
                dic[sen[i].sen].append(j)
        se.add(sen[i].sen)
    del(dic[""])
    return se,dic
#这个函数是用来测试是否可以生成spacy_DPtree的
#用来测试 不会在正式版出现    
def transfer2spacy(sen):
    for i in range(25,30):
        #vocab = Vocab(sen[i].word)
        #doc=Doc(vocab=vocab)
        #转到spacy出现了一个问题，现在的结构没有存第一个root元素，这不是很好，嘶
        #head = ['root']
        head = [None]
        tag = ['PU']
        dep = ['']
        word = ['ROOT']
        for j in range(len(sen[i].word)):
            #print(sen[i].dep[j],sen[i].idx[j])
            head.append(int(sen[i].dep[j]))
            word.append(sen[i].word[j])
            dep.append(sen[i].rel[j])
            tag.append(sen[i].tag[j])
        doc = Doc(nlp.vocab,words=word,tags=tag,deps=dep,heads=head)
        displacy.render(doc, style='dep', options = {'distance': 100})
        for token in doc:
            print(token.idx,token.text,token.tag_,token.dep_,token.head) # 解释标注
#生成svg图像 存储到指定位置
def generate_svg_image(path,svg):
    output_path = Path(path)
    with output_path.open("w", encoding="utf-8") as fh:
        fh.write(svg)

#生成graphiz类型的dependency_tree树
def generate_DPtree_graphiz(sen,st,dic,path):
    #sen是所有句子的集合，str是
    #我们需要为每一个句子生成一棵树，这里的话我们可以选择生成那个句子的树
    name = []
    for i in dic[st]:
        g = Digraph('依存树'+str(i),format='png')
        g.node(name='root')
        #需要让根节点指向0的位置
        for j in range(len(sen[i].word)):
            #如果需要添加词的属性需要加上下边这一行，否则就不需要
            #name.append(str(sen[i].word[j])+'|'+str(sen[i].tag[j]))
            g.node(sen[i].special_name[j],fontname="Microsoft YaHei")
        for j in range(len(sen[i].word)):
            if sen[i].rel[j]=='root':
                g.edge('root',sen[i].special_name[j],label=str(sen[i].dep[j]))
                
            else:
                g.edge(sen[i].Arc[sen[i].special_name[j]],sen[i].special_name[j],label=str(sen[i].rel[j]))
       # g.view()
        g.render(directory=path,view=True)
def generate_DPtree_spacy(sen,st,dic,path):
    #sen是所有句子的集合，str是
    #我们需要为每一个句子生成一棵树，这里的话我们可以选择生成那个句子的树
    for i in dic[st]:
        head = [None]
        tag = ['PU']
        dep = ['']
        word = ['ROOT']
        #需要让根节点指向0的位置
        for j in range(len(sen[i].word)):
            #print(sen[i].dep[j],sen[i].idx[j])
            head.append(int(sen[i].dep[j]))
            word.append(sen[i].word[j])
            dep.append(sen[i].rel[j])
            tag.append(sen[i].tag[j])
        doc = Doc(nlp.vocab,words=word,tags=tag,deps=dep,heads=head)
        svg = displacy.render(doc, style='dep', options = {'distance': 100})
        #print(type(svg)) #这里的输出是none 存在一个bug，后续会写一个博客来解释如何解决这个bug
        file = str(i)+'.svg'
        #utput_path = os.path.join(path,file)
        output_path = path +'/'+file
        print(output_path)
        #generate_svg_image(output_path,svg)
        for token in doc:
            print(token.idx,token.text,token.tag_,token.dep_,token.head) # 解释标注

In [44]:
file_path = "E:/baiduyunxiazai/zx/"
fila_name = "notMatch.alldata"
out_path = "E:/py基础/nlp_快速进入研究生状态/output_image"
dataset = []
data = Conll_8best_Read(dataset)
dataset = data.read_mul_file(file_path,fila_name)
sen = load_data_conll(dataset)
se,dic = find_sent(sen)
example = "只不过，在这之后，张小凡也感觉到，从绑在自己右手臂膀上的那个奇异法宝，却似乎散发着与烧火棍相反的，带着一丝温暖的气息，传进自己的身体。"
#generate_DPtree_graphiz(sen,example,dic,out_path)
generate_DPtree_spacy(sen,example,dic,out_path)

E:/baiduyunxiazai/zx/top1\notMatch.alldata
E:/baiduyunxiazai/zx/top2\notMatch.alldata
E:/baiduyunxiazai/zx/top3\notMatch.alldata
E:/baiduyunxiazai/zx/top4\notMatch.alldata
E:/baiduyunxiazai/zx/top5\notMatch.alldata
E:/baiduyunxiazai/zx/top6\notMatch.alldata
E:/baiduyunxiazai/zx/top7\notMatch.alldata
E:/baiduyunxiazai/zx/top8\notMatch.alldata
」


E:/py基础/nlp_快速进入研究生状态/output_image/161.svg
0 ROOT PU  ROOT
5 只 AD adv 不过
7 不过 AD adv 感觉
10 ， PU punc 感觉
12 在 P adv 感觉
14 这 PN att 之后
16 之后 LC pobj 在
19 ， PU punc 感觉
21 张小凡 NR subj 感觉
25 也 AD adv 感觉
27 感觉 VV root ROOT
30 到 VV cmp 感觉
32 ， PU punc 散发
34 从 P att 法宝
36 绑 VV pobj 从
38 在 P cmp 绑
40 自己 PN att 手
43 右 JJ att 手
45 手 NN att 臂膀
47 臂膀 NN att 上
50 上 LC pobj 在
52 的 DEG adjct 从
54 那 DT att 个
56 个 M att 法宝
58 奇异 JJ att 法宝
61 法宝 NN subj 散发
64 ， PU punc 散发
66 却 AD adv 散发
68 似乎 AD adv 散发
71 散发 VV obj 感觉
74 着 AS adjct 散发
76 与 P adv 相反
78 烧火棍 NR pobj 与
82 相反 VA de 的
85 的 DEC obj 散发
87 ， PU punc 散发
89 带 VV sasubj 散发
91 着 AS adjct 带
93 一 CD att 丝
95 丝 M att 气息
97 温暖 VA att 气息
100 的 DEC adjct 温暖
102 气息 NN obj 带
105 ， PU punc 传进
107 传进 VV pred 气息
110 自己 PN att 身体
113 的 DEG adjct 自己
115 身体 NN obj 传进
118 。 PU punc 散发


E:/py基础/nlp_快速进入研究生状态/output_image/562.svg
0 ROOT PU  ROOT
5 只 AD adv 不过
7 不过 AD adv 感觉
10 ， PU punc 感觉
12 在 P adv 感觉
14 这 PN att 之后
16 之后 LC pobj 在
19 ， PU punc 感觉
21 张小凡 NR subj 感觉
25 也 AD adv 感觉
27 感觉 VV root ROOT
30 到 VV cmp 感觉
32 ， PU punc 散发
34 从 P att 法宝
36 绑 VV pobj 从
38 在 P cmp 绑
40 自己 PN att 手
43 右 JJ att 手
45 手 NN att 臂膀
47 臂膀 NN att 上
50 上 LC pobj 在
52 的 DEG adjct 从
54 那 DT att 个
56 个 M att 法宝
58 奇异 JJ att 法宝
61 法宝 NN subj 散发
64 ， PU punc 散发
66 却 AD adv 散发
68 似乎 AD adv 散发
71 散发 VV dfsubj 感觉
74 着 AS adjct 散发
76 与 P adv 相反
78 烧火棍 NR pobj 与
82 相反 VA de 的
85 的 DEC obj 散发
87 ， PU punc 散发
89 带 VV sasubj 散发
91 着 AS adjct 带
93 一 CD att 丝
95 丝 M att 气息
97 温暖 VA att 气息
100 的 DEC adjct 温暖
102 气息 NN obj 带
105 ， PU punc 传进
107 传进 VV pred 气息
110 自己 PN att 身体
113 的 DEG adjct 自己
115 身体 NN obj 传进
118 。 PU punc 散发


E:/py基础/nlp_快速进入研究生状态/output_image/963.svg
0 ROOT PU  ROOT
5 只 AD adv 不过
7 不过 AD adv 感觉
10 ， PU punc 感觉
12 在 P adv 感觉
14 这 PN att 之后
16 之后 LC pobj 在
19 ， PU punc 感觉
21 张小凡 NR subj 感觉
25 也 AD adv 感觉
27 感觉 VV root ROOT
30 到 VV cmp 感觉
32 ， PU punc 散发
34 从 P att 法宝
36 绑 VV pobj 从
38 在 P coo 从
40 自己 PN att 手
43 右 JJ att 手
45 手 NN att 臂膀
47 臂膀 NN att 上
50 上 LC pobj 在
52 的 DEG adjct 从
54 那 DT att 个
56 个 M att 法宝
58 奇异 JJ att 法宝
61 法宝 NN subj 散发
64 ， PU punc 散发
66 却 AD adv 散发
68 似乎 AD adv 散发
71 散发 VV obj 感觉
74 着 AS adjct 散发
76 与 P adv 相反
78 烧火棍 NR pobj 与
82 相反 VA de 的
85 的 DEC obj 散发
87 ， PU punc 散发
89 带 VV sasubj 散发
91 着 AS adjct 带
93 一 CD att 丝
95 丝 M att 气息
97 温暖 VA att 气息
100 的 DEC adjct 温暖
102 气息 NN obj 带
105 ， PU punc 传进
107 传进 VV pred 气息
110 自己 PN att 身体
113 的 DEG adjct 自己
115 身体 NN obj 传进
118 。 PU punc 散发


E:/py基础/nlp_快速进入研究生状态/output_image/1364.svg
0 ROOT PU  ROOT
5 只 AD adv 不过
7 不过 AD adv 感觉
10 ， PU punc 感觉
12 在 P adv 感觉
14 这 PN att 之后
16 之后 LC pobj 在
19 ， PU punc 感觉
21 张小凡 NR subj 感觉
25 也 AD adv 感觉
27 感觉 VV root ROOT
30 到 VV cmp 感觉
32 ， PU punc 散发
34 从 P att 法宝
36 绑 VV pobj 从
38 在 P cmp 从
40 自己 PN att 手
43 右 JJ att 手
45 手 NN att 臂膀
47 臂膀 NN att 上
50 上 LC pobj 在
52 的 DEG adjct 从
54 那 DT att 个
56 个 M att 法宝
58 奇异 JJ att 法宝
61 法宝 NN subj 散发
64 ， PU punc 散发
66 却 AD adv 散发
68 似乎 AD adv 散发
71 散发 VV obj 感觉
74 着 AS adjct 散发
76 与 P adv 相反
78 烧火棍 NR pobj 与
82 相反 VA de 的
85 的 DEC obj 散发
87 ， PU punc 散发
89 带 VV sasubj 散发
91 着 AS adjct 带
93 一 CD att 丝
95 丝 M att 气息
97 温暖 VA att 气息
100 的 DEC adjct 温暖
102 气息 NN obj 带
105 ， PU punc 传进
107 传进 VV pred 气息
110 自己 PN att 身体
113 的 DEG adjct 自己
115 身体 NN obj 传进
118 。 PU punc 散发


E:/py基础/nlp_快速进入研究生状态/output_image/1765.svg
0 ROOT PU  ROOT
5 只 AD adv 不过
7 不过 AD adv 感觉
10 ， PU punc 感觉
12 在 P adv 感觉
14 这 PN att 之后
16 之后 LC pobj 在
19 ， PU punc 感觉
21 张小凡 NR subj 感觉
25 也 AD adv 感觉
27 感觉 VV root ROOT
30 到 VV cmp 感觉
32 ， PU punc 散发
34 从 P att 法宝
36 绑 VV pobj 从
38 在 P cmp 绑
40 自己 PN att 手
43 右 JJ att 手
45 手 NN att 臂膀
47 臂膀 NN att 上
50 上 LC pobj 从
52 的 DEG adjct 从
54 那 DT att 个
56 个 M att 法宝
58 奇异 JJ att 法宝
61 法宝 NN subj 散发
64 ， PU punc 散发
66 却 AD adv 散发
68 似乎 AD adv 散发
71 散发 VV obj 感觉
74 着 AS adjct 散发
76 与 P adv 相反
78 烧火棍 NR pobj 与
82 相反 VA de 的
85 的 DEC obj 散发
87 ， PU punc 散发
89 带 VV sasubj 散发
91 着 AS adjct 带
93 一 CD att 丝
95 丝 M att 气息
97 温暖 VA att 气息
100 的 DEC adjct 温暖
102 气息 NN obj 带
105 ， PU punc 传进
107 传进 VV pred 气息
110 自己 PN att 身体
113 的 DEG adjct 自己
115 身体 NN obj 传进
118 。 PU punc 散发


E:/py基础/nlp_快速进入研究生状态/output_image/2166.svg
0 ROOT PU  ROOT
5 只 AD adv 不过
7 不过 AD adv 感觉
10 ， PU punc 感觉
12 在 P adv 感觉
14 这 PN att 之后
16 之后 LC pobj 在
19 ， PU punc 感觉
21 张小凡 NR subj 感觉
25 也 AD adv 感觉
27 感觉 VV root ROOT
30 到 VV cmp 感觉
32 ， PU punc 散发
34 从 P att 法宝
36 绑 VV pobj 从
38 在 P cmp 绑
40 自己 PN att 手
43 右 JJ att 手
45 手 NN att 臂膀
47 臂膀 NN att 上
50 上 LC pobj 从
52 的 DEG adjct 从
54 那 DT att 个
56 个 M att 法宝
58 奇异 JJ att 法宝
61 法宝 NN subj 散发
64 ， PU punc 散发
66 却 AD adv 散发
68 似乎 AD adv 散发
71 散发 VV dfsubj 感觉
74 着 AS adjct 散发
76 与 P adv 相反
78 烧火棍 NR pobj 与
82 相反 VA de 的
85 的 DEC obj 散发
87 ， PU punc 散发
89 带 VV sasubj 散发
91 着 AS adjct 带
93 一 CD att 丝
95 丝 M att 气息
97 温暖 VA att 气息
100 的 DEC adjct 温暖
102 气息 NN obj 带
105 ， PU punc 传进
107 传进 VV pred 气息
110 自己 PN att 身体
113 的 DEG adjct 自己
115 身体 NN obj 传进
118 。 PU punc 散发


E:/py基础/nlp_快速进入研究生状态/output_image/2567.svg
0 ROOT PU  ROOT
5 只 AD adv 不过
7 不过 AD adv 感觉
10 ， PU punc 感觉
12 在 P adv 感觉
14 这 PN att 之后
16 之后 LC pobj 在
19 ， PU punc 感觉
21 张小凡 NR subj 感觉
25 也 AD adv 感觉
27 感觉 VV root ROOT
30 到 VV cmp 感觉
32 ， PU punc 散发
34 从 P att 法宝
36 绑 VV pobj 从
38 在 P coo 从
40 自己 PN att 手
43 右 JJ att 手
45 手 NN att 臂膀
47 臂膀 NN att 上
50 上 LC pobj 从
52 的 DEG adjct 从
54 那 DT att 个
56 个 M att 法宝
58 奇异 JJ att 法宝
61 法宝 NN subj 散发
64 ， PU punc 散发
66 却 AD adv 散发
68 似乎 AD adv 散发
71 散发 VV obj 感觉
74 着 AS adjct 散发
76 与 P adv 相反
78 烧火棍 NR pobj 与
82 相反 VA de 的
85 的 DEC obj 散发
87 ， PU punc 散发
89 带 VV sasubj 散发
91 着 AS adjct 带
93 一 CD att 丝
95 丝 M att 气息
97 温暖 VA att 气息
100 的 DEC adjct 温暖
102 气息 NN obj 带
105 ， PU punc 传进
107 传进 VV pred 气息
110 自己 PN att 身体
113 的 DEG adjct 自己
115 身体 NN obj 传进
118 。 PU punc 散发


E:/py基础/nlp_快速进入研究生状态/output_image/2968.svg
0 ROOT PU  ROOT
5 只 AD adv 不过
7 不过 AD adv 感觉
10 ， PU punc 感觉
12 在 P adv 感觉
14 这 PN att 之后
16 之后 LC pobj 在
19 ， PU punc 感觉
21 张小凡 NR subj 感觉
25 也 AD adv 感觉
27 感觉 VV root ROOT
30 到 VV cmp 感觉
32 ， PU punc 散发
34 从 P att 法宝
36 绑 VV pobj 从
38 在 P cmp 从
40 自己 PN att 手
43 右 JJ att 手
45 手 NN att 臂膀
47 臂膀 NN att 上
50 上 LC pobj 从
52 的 DEG adjct 从
54 那 DT att 个
56 个 M att 法宝
58 奇异 JJ att 法宝
61 法宝 NN subj 散发
64 ， PU punc 散发
66 却 AD adv 散发
68 似乎 AD adv 散发
71 散发 VV obj 感觉
74 着 AS adjct 散发
76 与 P adv 相反
78 烧火棍 NR pobj 与
82 相反 VA de 的
85 的 DEC obj 散发
87 ， PU punc 散发
89 带 VV sasubj 散发
91 着 AS adjct 带
93 一 CD att 丝
95 丝 M att 气息
97 温暖 VA att 气息
100 的 DEC adjct 温暖
102 气息 NN obj 带
105 ， PU punc 传进
107 传进 VV pred 气息
110 自己 PN att 身体
113 的 DEG adjct 自己
115 身体 NN obj 传进
118 。 PU punc 散发


In [7]:
print(dic["只不过，在这之后，张小凡也感觉到，从绑在自己右手臂膀上的那个奇异法宝，却似乎散发着与烧火棍相反的，带着一丝温暖的气息，传进自己的身体。"])
print(sen[161].sen)

[161, 562, 963, 1364, 1765, 2166, 2567, 2968]
只不过，在这之后，张小凡也感觉到，从绑在自己右手臂膀上的那个奇异法宝，却似乎散发着与烧火棍相反的，带着一丝温暖的气息，传进自己的身体。


In [8]:


#生成svg图像 没仔细研究
def generate_svg_image(path,doc):
    output_path = Path("yourpath/.svg")
    svg = displacy.render(doc, style='dep')
    with output_path.open("w", encoding="utf-8") as fh:
        fh.write(svg)


In [9]:
transfer2spacy(sen)

0 ROOT PU  ROOT
5 」 PU root ROOT


0 ROOT PU  ROOT
5 第一 OD att 章
8 章 M att 隐忧
10 隐忧 NN root ROOT


0 ROOT PU  ROOT
5 」 PU root ROOT


0 ROOT PU  ROOT
5 只 AD adv 有
7 有 VE root ROOT
9 风声 NN obj 有
12 ！ PU punc 有


0 ROOT PU  ROOT
5 」 PU root ROOT
